In [159]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go


from cointegration.cointegration_system import CointegrationTradingSystem
from cointegration.cointegration_optimization import OptimizeCointegrationTradingSystem

In [160]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [161]:
%%R
# Install required packages
if (require("dplyr") == FALSE) {
  install.packages("dplyr")
  library(dplyr)
}
if (require("zoo") == FALSE) {
  install.packages("zoo")
  library(zoo)
}

if (require("forecast") == FALSE) {
  install.packages("forecast")
  library(forecast)
}
if (require("tseries") == FALSE) {
  install.packages("tseries")
  library(tseries)
}
if (require("moments") == FALSE) {
  install.packages("moments")
  library(moments)
}
if (require("tibble") == FALSE) {
  install.packages("tibble")
  library(tibble)
}
if (require("urca") == FALSE) {
  install.packages("urca")
  library(urca)
}
if (require("tsDyn") == FALSE) {
  install.packages("tsDyn")
  library(tsDyn)
}

In [166]:
hub1_name = "ttf"
hub2_name = "the"
validation_size = 250
test_size = 250
window_size = 5

In [167]:
%R -i hub1_name
%R -i hub2_name
%R -i validation_size
%R -i test_size

In [168]:
%%R
hubs <- list(
    "nbp" = read.csv("../data/interpolated/nbp_close_interpolated.csv"),
    "peg" = read.csv("../data/interpolated/peg_close_interpolated.csv"),
    "the" = read.csv("../data/interpolated/the_close_interpolated.csv"),
    "ttf" = read.csv("../data/interpolated/ttf_close_interpolated.csv"),
    "ztp" = read.csv("../data/interpolated/ztp_close_interpolated.csv")
)


In [169]:
%%R
hub1 <- hubs[[hub1_name]]
hub2 <- hubs[[hub2_name]]

aics <- c()
bics <- c()



hubs <- data.frame(hub1 = hub1$CLOSE, hub2 = hub2$CLOSE)

train_size <- nrow(hubs) - test_size + 1
train <- hubs[1:train_size,]
max_lag <- 20
for (p in 1:max_lag) {
vecm <- VECM(train, lag = p,  r = 1, include = "const", estim = "ML")
aics <- c(aics, AIC(vecm))
bics <- c(bics, BIC(vecm))
}
lags <- which.min(bics)
print(lags)


jotest <- ca.jo(train, type="trace", ecdet="none", spec="transitory", K = lags)

summary(jotest)

[1] 5

###################### 
# Johansen-Procedure # 
###################### 

Test type: trace statistic , with linear trend 

Eigenvalues (lambda):
[1] 0.043417338 0.004061242

Values of teststatistic and critical values of test:

          test 10pct  5pct  1pct
r <= 1 |  5.25  6.50  8.18 11.65
r = 0  | 62.56 15.66 17.95 23.52

Eigenvectors, normalised to first column:
(These are the cointegration relations)

           hub1.l1    hub2.l1
hub1.l1  1.0000000  1.0000000
hub2.l1 -0.9978803 -0.8443134

Weights W:
(This is the loading matrix)

         hub1.l1     hub2.l1
hub1.d 0.5970060 -0.04519086
hub2.d 0.7976074 -0.04404850



In [170]:
%%R

hs <- tsDyn:::TVECM.HStest(train, lag = lags, ngridTh = 100)


In [171]:
%%R
summary(hs)

## Test of linear versus threshold cointegration of Hansen and Seo (2002) ##

Test Statistic:	 46.25808	(Maximized for threshold value: 0.04513681 )
P-Value:	 0.01 		( Fixed regressor bootstrap )

Critical values:
    0.90%   0.95%   0.99%
 37.11437 38.3304 44.6556
Number of bootstrap replications:	 100 

Cointegrating value (estimated under restricted linear model): -0.9979222 


In [172]:
%%R
beta = (-1) * jotest@V[2,1]

In [173]:
%R -o beta

In [174]:
octs = OptimizeCointegrationTradingSystem(hub1_name, hub2_name, beta.item(), validation_size, test_size, window_size)

In [175]:
est_profit, best_rolling_window, best_lower_threshold, best_upper_threshold = octs.study(rolling_window_range=[5,60,5], 
                                                                                         lower_threshold_range=[0.2,2,0.2], 
                                                                                         upper_threshold_range=[2,4,0.5], 
                                                                                         criteria="profit", 
                                                                                         min_trades=10, 
                                                                                         verbose=True, 
                                                                                         plot=True)

Best rolling window: 55.00, Best lower threshold: 0.80, Best upper threshold: 3.50
Profit: 19.61

Mean returns: 0.06%
Standard deviation of returns: 0.01%
Sharpe ratio: 10.74
Mean returns with transaction costs: -0.00%
Confidence interval (returns): -0.01% - 0.14%
Confidence interval (returns with transaction costs): -0.00% - -0.00%

Win rate for returns: 36.00%
No trade rate for returns: 33.20%
Loss rate for returns: 30.80%

Win rate for returns with transaction costs: 0.00%
No trade rate for returns with transaction costs: 33.20%
Loss rate for returns with transaction costs: 66.80%


In [176]:
cts = CointegrationTradingSystem(hub1_name, hub2_name, 0.9, test_size, window_size)

In [177]:
cts.run_trading_system(rolling_window=best_rolling_window, lower_threshold=best_lower_threshold, upper_threshold=best_upper_threshold, verbose=True, plot=True)

Profit: -1.28

Mean returns: -0.01%
Standard deviation of returns: 0.00%
Sharpe ratio: -10.95
Mean returns with transaction costs: 0.00%
Confidence interval (returns): -0.01% - 0.00%
Confidence interval (returns with transaction costs): 0.00% - 0.00%

Win rate for returns: 1.60%
No trade rate for returns: 96.00%
Loss rate for returns: 2.40%

Win rate for returns with transaction costs: 4.00%
No trade rate for returns with transaction costs: 96.00%
Loss rate for returns with transaction costs: 0.00%
